In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import re
import h5py
import math
import datetime
import torch

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly_resampler


def plot_traces(traces, sr, dtype='acc'):
    n_samples = len(traces)
    timesteps = np.linspace(0, math.ceil(n_samples / sr), n_samples, endpoint=False)

    if dtype == 'acc':
        g = 9.81
        traces = traces / g

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

    colors = ['red', 'green', 'blue']
    axis_labels = ['X', 'Y', 'Z']

    for i in range(3):
        fig.add_trace(
            go.Scattergl(
                x=timesteps,
                y=traces[:, i],
                name=f'{axis_labels[i]}',
                line=dict(color=colors[i])
            ),
            row=i+1, col=1
        )
        fig.update_yaxes(
            title_text=f"{'Acceleration (G)' if dtype == 'acc' else 'Angular Velocity (°/s)'}",
            row=i+1, col=1)

    fig.update_layout(
        title_text=f"{'Accelerometer' if dtype == 'acc' else 'Gyroscope'} Traces",
        height=700
    )
    fig.update_xaxes(title_text="Time (s)", row=3, col=1)

    fig = plotly_resampler.FigureWidgetResampler(fig)

    return fig

In [16]:
with h5py.File('/data/IDLab/aar_foundation_models/processed_data/calves/cow_1444_ds_19.hdf5', 'r') as f:
    sr = f.attrs['sr']
    X_acc = f['acc'][:]
    # X_gyr = f['gyr'][:]


In [17]:
plot_traces(X_acc, sr, dtype='acc')

FigureWidgetResampler({
    'data': [{'line': {'color': 'red'},
              'name': 'X',
              'type': 'scattergl',
              'uid': 'b7c7bab7-34f8-4bf9-8f14-61dc38988cba',
              'x': array([0.        , 0.04137931, 0.08275862, 0.12413793, 0.16551724, 0.20689655,
                          0.24827586, 0.28965517, 0.33103448, 0.37241379, 0.4137931 , 0.45517241,
                          0.49655172, 0.53793103, 0.57931034, 0.62068966, 0.66206897, 0.70344828,
                          0.74482759, 0.7862069 , 0.82758621, 0.86896552, 0.91034483, 0.95172414,
                          0.99310345, 1.03448276, 1.07586207, 1.11724138, 1.15862069, 1.2       ,
                          1.24137931, 1.28275862, 1.32413793, 1.36551724, 1.40689655, 1.44827586,
                          1.48965517, 1.53103448, 1.57241379, 1.6137931 , 1.65517241, 1.69655172,
                          1.73793103, 1.77931034, 1.82068966, 1.86206897, 1.90344828, 1.94482759,
                          1.9

In [65]:
processed_data_root = Path('/data/IDLab/aar_foundation_models/processed_data')
ds_info_dict = {}
for ds_path in processed_data_root.glob('*'):
    individual_animals = set()
    total_seconds = 0
    for hdf5_path in ds_path.glob('*.hdf5'):
        info_tuple = hdf5_path.stem.split('_')
        species = info_tuple[0]
        animal_id = int(info_tuple[1])
        ds_index = int(info_tuple[3])

        individual_animals.add(animal_id)
        with h5py.File(hdf5_path, 'r') as f:
            sr = f.attrs['sr']
            data_type = next(filter(lambda e: e.startswith('acc'), f.keys()))
            total_seconds += len(f[data_type]) / sr
    ds_info_dict[ds_path.name] = (total_seconds, len(individual_animals))

In [66]:
for ds_name, (total_seconds, total_individuals) in ds_info_dict.items():
    seconds_td = datetime.timedelta(seconds=total_seconds)
    print(f"Dataset '{ds_name}': {seconds_td} ({total_individuals} individuals)")

Dataset 'calves': 1 day, 3:24:46.480000 (30 individuals)
Dataset 'cats': 0:54:10.275000 (9 individuals)
Dataset 'horsing_around': 16 days, 13:56:11.990000 (18 individuals)
Dataset 'idlab_foaling_2019': 135 days, 7:14:15.120000 (18 individuals)
Dataset 'kamminga_goats': 1 day, 15:27:13.980000 (5 individuals)
Dataset 'waves_equine_activities': 6:58:36.220000 (14 individuals)


In [1]:
min(i for i in range(10))

0

In [4]:
l = torch.nn.Linear(32, 512)
x = torch.randn((64, 100, 8, 32))
l(x).shape

torch.Size([64, 100, 8, 512])